# <b>1. Model setup</b>

In [ ]:
import os

# Download model weight
if not os.path.exists('yolov7.pt'):
  # Download model weight
  ! wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt -O yolov7.pt


# Create the model
from traffic_detector import TrafficDetector
detector = TrafficDetector()
detector.set_conf('weights', 'yolov7.pt')
detector.setup_model()

# <b>2. Test on one image</b>

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams["figure.figsize"] = (40,10)


img = cv2.imread('data/vehicle_test.png')
output=detector.detect(img)
# cv2.imread: BGR -> plt.imshow(rgb)
plt.subplot(121)
plt.imshow(img[:,:,::-1])
plt.title('Input image')
plt.axis('off')

plt.subplot(122)
plt.imshow(output[:,:,::-1])
plt.title('Detection result')
plt.axis('off')
plt.show()

# <b>3. Test on a video</b>

You can click on the button to switch between diplaying the original video frame and the detection results.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

# Detect button
# ================
detect_button = widgets.ToggleButton(
    value=False,
    description='Vehicle Detection',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)


# Display function
# ================
def view(button, video_source):
    cap = cv2.VideoCapture(video_source)
    display_handle=display(None, display_id=True)
    i = 0
    while True:
        _, frame = cap.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        if detect_button.value==True:
            # do detection
            frame = detector.detect(frame, output_ratio=2)
            button.description = 'Original Frame'
        else:
            button.description = 'Vehicle Detection'
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(Image(data=frame.tobytes()))
                
    # close the video and the display
    cap.release()
    display_handle.update(None)

            
# Run
# ================
display(detect_button)
video_source = 'data/vehicle_test.mp4'
thread = threading.Thread(target=view, args=(detect_button, video_source))
thread.start()

# <b>4. Test on Live camera stream</b>

In [ ]:
!pip install --force-reinstall git+https://github.com/ytdl-org/youtube-dl.git@master
!pip install --force-reinstall git+https://github.com/donglaiw/pafy.git@develop

In [ ]:
import pafy

# get live traffic camera source
url = "https://youtu.be/5_XSYlAfJZM"
video = pafy.new(url)
best = video.getbest(preftype="mp4")


# Run
# ================
video_source = best.url
display(detect_button)
thread = threading.Thread(target=view, args=(detect_button, video_source))
thread.start()